# Booz Allen Spring 2025 Codefest: Challenge 1 
### Isaiah Byrd, Kyler Gelissen, David Ameh

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumn2u/garbage-classification-v2")

In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import torch as th
import cv2
import json
import random
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Dataset

In [ ]:
path = "/kaggle/input/garbage-classification-v2/garbage-dataset"

In [ ]:
#Set divice to GPU if available
divice = th.device("cuda" if th.cuda.is_available() else "cpu")

#Change the memory fraction to limit the GPU memory usage
#Set the memory fraction to 60% of the total GPU memory
memoryFraction = 0.6

#Limits the GPU memory usage to 60% 
if(divice.type == "cuda"):
    print("GPU is available")
    th.cuda.set_memory_fraction(memoryFraction, divice=divice.index)